In [ ]:
!pip install tensorflow==2.17.1 nibabel matplotlib scikit-learn faiss-cpu


In [ ]:
import os, glob, random, tempfile, requests
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from collections import Counter

import faiss


In [ ]:
import os, requests

base = ("https://raw.githubusercontent.com/muschellij2/open_ms_data/"
        "master/cross_sectional/coregistered_resampled")

patients = [f"patient{i:02d}" for i in range(1, 31)]

modalities = {
    "FLAIR": "FLAIR.nii.gz",
    "Brain Mask": "brainmask.nii.gz",
    "Lesion Mask": "consensus_gt.nii.gz"
}

out_dir = "ms_data_resampled_labeled"
os.makedirs(out_dir, exist_ok=True)

def download_and_log(url, out):
    if os.path.exists(out):
        print("↪ Already exists:", out)
        return True
    r = requests.get(url, stream=True)
    if r.ok:
        with open(out, "wb") as f:
            for chunk in r.iter_content(1 << 20):  # 1 MB chunks
                f.write(chunk)
        print("✅", out)
        return True
    else:
        print("❌ Failed:", url, "status:", r.status_code)
        return False

downloaded = []
for pid in patients:
    success = True
    for label, fname in modalities.items():
        url = f"{base}/{pid}/{fname}"
        out = os.path.join(out_dir, f"{pid}_{label.replace(' ', '_')}.nii.gz")
        if not download_and_log(url, out):
            success = False
    if success:
        downloaded.append(pid)

print(f"\nDownload complete. Successful patients: {len(downloaded)} / {len(patients)}")


In [ ]:
DATA_DIR = "ms_data_resampled_labeled"
patients = sorted(list({os.path.basename(p).split("_")[0] for p in glob.glob(os.path.join(DATA_DIR, "*_FLAIR.nii.gz"))}))
print("Patients found:", len(patients))

IMG_SIZE = (299, 299)
SLICES_PER_PATIENT = 12

def slice_indices_centered(n_slices, k=SLICES_PER_PATIENT):
    mid = n_slices // 2
    start = max(0, mid - k//2)
    end = min(n_slices, start + k)
    start = max(0, end - k)
    return list(range(start, end))

def preprocess_slice(slice_img):
    s = slice_img.astype(np.float32)
    mn, mx = np.percentile(s, 0.5), np.percentile(s, 99.5)
    s = np.clip((s - mn) / max(mx - mn, 1e-6), 0, 1)
    s = tf.image.resize(s[..., None], IMG_SIZE)
    s = tf.image.grayscale_to_rgb(s)   # shape (299,299,3)
    return s.numpy()

slice_records = []
for pid in patients:
    flair_path = os.path.join(DATA_DIR, f"{pid}_FLAIR.nii.gz")
    mask_path  = os.path.join(DATA_DIR, f"{pid}_Lesion_Mask.nii.gz")
    if not (os.path.exists(flair_path) and os.path.exists(mask_path)):
        continue
    flair = nib.load(flair_path)
    mask  = nib.load(mask_path)
    nz = flair.shape[2]
    idxs = slice_indices_centered(nz, SLICES_PER_PATIENT)
    for z in idxs:
        mask_slice = mask.get_fdata()[:, :, z]
        label = 1 if np.count_nonzero(mask_slice) > 0 else 0
        slice_records.append((pid, z, label))
print("Total slices prepared (approx):", len(slice_records))


In [ ]:
random.seed(42)
train_p, test_p = train_test_split(patients, test_size=0.2, random_state=42)
train_p, val_p = train_test_split(train_p, test_size=0.2, random_state=42)
print("Patients split -> train:", len(train_p), "val:", len(val_p), "test:", len(test_p))

def dataset_for_patients(pids, batch=16, augment=False, shuffle=True):
    def gen():
        for pid in pids:
            flair_path = os.path.join(DATA_DIR, f"{pid}_FLAIR.nii.gz")
            mask_path  = os.path.join(DATA_DIR, f"{pid}_Lesion_Mask.nii.gz")
            flair = nib.load(flair_path).get_fdata()
            mask  = nib.load(mask_path).get_fdata()
            nz = flair.shape[2]
            idxs = slice_indices_centered(nz, SLICES_PER_PATIENT)
            for z in idxs:
                s = preprocess_slice(flair[:, :, z])
                lbl = 1 if np.count_nonzero(mask[:, :, z]) > 0 else 0
                yield s.astype(np.float32), np.float32(lbl)
    out_sig = (
        tf.TensorSpec(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )
    ds = tf.data.Dataset.from_generator(gen, output_signature=out_sig)
    if shuffle:
        ds = ds.shuffle(1024, reshuffle_each_iteration=True)
    if augment:
        def aug(x, y):
            x = tf.image.random_flip_left_right(x)
            x = tf.image.random_flip_up_down(x)
            x = tf.image.random_brightness(x, 0.05)
            return x, y
        ds = ds.map(aug, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = dataset_for_patients(train_p, batch=16, augment=True, shuffle=True)
val_ds   = dataset_for_patients(val_p,   batch=16, augment=False, shuffle=False)
test_ds  = dataset_for_patients(test_p,  batch=16, augment=False, shuffle=False)


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

base = InceptionV3(weights="imagenet", include_top=False, input_shape=(299,299,3))
for l in base.layers:
    l.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base.input, outputs=out)
model.compile(
    optimizer=Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.BinaryAccuracy(name="acc"), tf.keras.metrics.AUC(name="auc")]
)

model.summary()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", patience=5, mode="max", restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=4, callbacks=callbacks)

# Define the save path in your Google Drive
save_path = "/content/drive/MyDrive/model.h5"

model.save(save_path)
print(f"Model saved to: {save_path}")

In [ ]:
# collect predictions and true labels on test set
y_true, y_prob = [], []
for x_batch, y_batch in test_ds:
    probs = model.predict(x_batch, verbose=0).ravel()
    y_prob.append(probs)
    y_true.append(y_batch.numpy())
y_true = np.concatenate(y_true)
y_prob = np.concatenate(y_prob)

y_pred = (y_prob >= 0.5).astype(int)
print("ROC AUC:", roc_auc_score(y_true, y_prob))
print(classification_report(y_true, y_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))


In [ ]:

def patient_probs(pid):
    flair_path = os.path.join(DATA_DIR, f"{pid}_FLAIR.nii.gz")
    mask_path  = os.path.join(DATA_DIR, f"{pid}_Lesion_Mask.nii.gz")
    flair = nib.load(flair_path).get_fdata()
    mask  = nib.load(mask_path).get_fdata()
    idxs = slice_indices_centered(flair.shape[2], SLICES_PER_PATIENT)
    probs = []
    for z in idxs:
        s = preprocess_slice(flair[:, :, z])
        p = float(model.predict(s[None,...], verbose=0)[0,0])
        probs.append((z, p, int(np.count_nonzero(mask[:,:,z])>0)))
    return probs

# Evaluate on test patients
scan_y_true, scan_y_prob = [], []
for pid in test_p:
    probs = patient_probs(pid)
    per_slice_probs = [p for (_,p,_) in probs]
    scan_prob_max = max(per_slice_probs)
    scan_prob_mean = float(np.mean(per_slice_probs))
    gt = int(any([lab for (_,_,lab) in probs]))
    scan_y_true.append(gt)
    scan_y_prob.append(scan_prob_max)   # use max for demo

print("Scan ROC AUC:", roc_auc_score(scan_y_true, scan_y_prob))
scan_pred = [1 if p>=0.5 else 0 for p in scan_y_prob]
print(classification_report(scan_y_true, scan_pred, digits=4))
print("Confusion matrix (scan-level):\n", confusion_matrix(scan_y_true, scan_pred))


In [ ]:
def show_patient(pid):
    probs = patient_probs(pid)
    flair_path = os.path.join(DATA_DIR, f"{pid}_FLAIR.nii.gz")
    mask_path  = os.path.join(DATA_DIR, f"{pid}_Lesion_Mask.nii.gz")
    flair = nib.load(flair_path).get_fdata()
    mask  = nib.load(mask_path).get_fdata()>0
    plt.figure(figsize=(15,3))
    for i,(z,p,gt) in enumerate(probs):
        sl = flair[:,:,z]
        plt.subplot(1,len(probs), i+1)
        plt.imshow(sl.T, cmap="gray", origin="lower")
        if gt:
            plt.contour(mask[:,:,z].T, levels=[0.5], colors='r')
        plt.title(f"z={z}\np={p:.2f}")
        plt.axis("off")
    plt.suptitle(f"Patient {pid} - slice probs")
    plt.show()

for pid in test_p[:3]:
    show_patient(pid)


In [ ]:
!pip install gradio


In [ ]:
import tensorflow as tf

# Define the path to your saved model in Google Drive
load_path = "/content/drive/MyDrive/model.h5"

# Load the model
model = tf.keras.models.load_model(load_path)

# Now you can use the loaded_model for predictions or further training
# For example, you can replace the 'model' variable in your Gradio interface cell
# with 'loaded_model' to use the loaded model.

In [ ]:
def predict_mri(file):
    # Load the MRI file (.nii.gz)
    img = nib.load(file.name).get_fdata()
    img_slice = img[:, :, img.shape[2] // 2]  # take middle slice

    # Preprocess to model input shape
    img_resized = np.resize(img_slice, (299, 299, 1)) / np.max(img_slice)
    img_batch = np.expand_dims(img_resized, axis=0)

    # Run prediction
    pred = model.predict(img_batch)[0][0]
    label = "🧠 Lesion Detected" if pred > 0.5 else "✅ Normal"

    # Optional: visualize the MRI slice
    plt.imshow(img_slice, cmap="gray")
    plt.axis("off")
    plt.title(label)
    plt.show()

    return label


In [ ]:
demo = gr.Interface(
    fn=predict_mri,
    inputs=gr.File(label="Upload MRI file (.nii.gz)"),
    outputs=gr.Textbox(label="Prediction"),
    title="MRI Brain Lesion Detection Demo",
    description="Upload an MRI scan to test the trained CNN model for MS lesion detection."
)

demo.launch()
